In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras import metrics  

from tensorflow.keras import layers
from keras.applications import resnet
class SiameseModel(Model):
    """The Siamese Network model with a custom training and testing loops.

    Computes the triplet loss using the three embeddings produced by the
    Siamese Network.

    The triplet loss is defined as:
       L(A, P, N) = max(‖f(A) - f(P)‖² - ‖f(A) - f(N)‖² + margin, 0)
    """

    def __init__(self, siamese_network, margin=0.5,**kwargs):
        super(SiameseModel, self).__init__(**kwargs)
        self.siamese_network = siamese_network
        self.margin = margin
        self.loss_tracker = metrics.Mean(name="loss")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        # GradientTape is a context manager that records every operation that
        # you do inside. We are using it here to compute the loss so we can get
        # the gradients and apply them using the optimizer specified in
        # `compile()`.
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)

        # Storing the gradients of the loss function with respect to the
        # weights/parameters.
        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)

        # Applying the gradients on the model using the specified optimizer
        self.optimizer.apply_gradients(
            zip(gradients, self.siamese_network.trainable_weights)
        )

        # Let's update and return the training loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)

        # Let's update and return the loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _compute_loss(self, data):
        # The output of the network is a tuple containing the distances
        # between the anchor and the positive example, and the anchor and
        # the negative example.
        ap_distance, an_distance = self.siamese_network(data)

        # Computing the Triplet Loss by subtracting both distances and
        # making sure we don't get a negative value.
        loss = ap_distance - an_distance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss

    @property
    def metrics(self):
        # We need to list our metrics here so the `reset_states()` can be
        # called automatically.
        return [self.loss_tracker]

    # Add get_config and from_config for serialization
    def get_config(self):
        config = super(SiameseModel, self).get_config()
        config.update({
            "siamese_network": self.siamese_network.get_config() if hasattr(self.siamese_network, "get_config") else None
        })
        return config
        
    @classmethod
    def from_config(cls, config):
        siamese_network = config.pop("siamese_network")  # Extract custom objects
        return cls(siamese_network=siamese_network, **config)



In [ ]:
target_shape = (200,200)
class DistanceLayer(layers.Layer):
    """
    This layer is responsible for computing the distance between the anchor
    embedding and the positive embedding, and the anchor embedding and the
    negative embedding.
    """

    def __init__(self, **kwargs):
        super(DistanceLayer, self).__init__(**kwargs)

    def call(self, anchor, positive, negative):
        # 使用 TensorFlow 的 reduce_sum 来计算距离
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
        return (ap_distance, an_distance)


In [ ]:
#target_shape = (200,200)  # 假设图像是 RGB 格式，大小为 200x200

base_cnn = resnet.ResNet50(
    weights="imagenet", input_shape=target_shape + (3,), include_top=False
)
#print(base_cnn.layers)
flatten = layers.Flatten()(base_cnn.output)
dense1 = layers.Dense(512, activation="relu")(flatten)
dense1 = layers.BatchNormalization()(dense1)
dense2 = layers.Dense(256, activation="relu")(dense1)
dense2 = layers.BatchNormalization()(dense2)
output = layers.Dense(256)(dense2)

embedding = Model(base_cnn.input, output, name="Embedding")

trainable = False
#adjust the number of trainable layers
for layer in base_cnn.layers:
    if layer.name == "conv5_block1_out":
        trainable = True
    layer.trainable = trainable

In [ ]:

anchor_input = layers.Input(name="anchor", shape=target_shape + (3,))
positive_input = layers.Input(name="positive", shape=target_shape + (3,))
negative_input = layers.Input(name="negative", shape=target_shape + (3,))

distances = DistanceLayer()(
    embedding(resnet.preprocess_input(anchor_input)),
    embedding(resnet.preprocess_input(positive_input)),
    embedding(resnet.preprocess_input(negative_input)),
)

# 假设你已经定义了一个 SiameseNetwork 类
siamese_network = Model(
    inputs=[anchor_input, positive_input, negative_input], outputs=distances
)

# 然后将它作为参数传递给 SiameseModel
siamese_model = SiameseModel(siamese_network=siamese_network)

input_shape = (None, 200, 200, 3)  # 这里你需要使用与你的数据一致的输入形状
siamese_model.build(input_shape)
# 如果加载权重，可以这样做
siamese_model.load_weights('siamese_18k_epoch20_trainable1_weights.weights.h5')

# 查看加载后的模型结构
siamese_model.summary()

In [ ]:
#test two dances
#for testing dataset
from pathlib import Path
cache_dir = Path(Path.home()) / "autodl-tmp"

import os

In [ ]:
pip install python-docx


In [ ]:
#unzip all the test data
!unzip -oq test6a_outcome_integrate.zip -d $cache_dir
!unzip -oq test6c_outcome_integrate.zip -d $cache_dir
!unzip -oq test6e_outcome_integrate.zip -d $cache_dir
!unzip -oq test6f_outcome_integrate.zip -d $cache_dir
!unzip -oq test6g_outcome_integrate.zip -d $cache_dir
!unzip -oq test6h_outcome_integrate.zip -d $cache_dir
!unzip -oq test6i_outcome_integrate.zip -d $cache_dir
!unzip -oq test6j_outcome_integrate.zip -d $cache_dir
!unzip -oq test6k_outcome_integrate.zip -d $cache_dir
!unzip -oq test6l_outcome_integrate.zip -d $cache_dir
!unzip -oq test3d_outcome_integrate.zip -d $cache_dir
!unzip -oq test3e_outcome_integrate.zip -d $cache_dir
!unzip -oq test3g_outcome_integrate.zip -d $cache_dir
!unzip -oq test3h_outcome_integrate.zip -d $cache_dir
!unzip -oq test4c_outcome_integrate.zip -d $cache_dir
!unzip -oq test4d_outcome_integrate.zip -d $cache_dir
!unzip -oq test4e_outcome_integrate.zip -d $cache_dir
!unzip -oq test4f_outcome_integrate.zip -d $cache_dir
!unzip -oq test5a_outcome_integrate.zip -d $cache_dir
!unzip -oq test5b_outcome_integrate.zip -d $cache_dir
!unzip -oq test5c_outcome_integrate.zip -d $cache_dir
!unzip -oq test5d_outcome_integrate.zip -d $cache_dir
!unzip -oq test5g_outcome_integrate.zip -d $cache_dir
!unzip -oq test5h_outcome_integrate.zip -d $cache_dir
!unzip -oq test5k_outcome_integrate.zip -d $cache_dir
!unzip -oq test5l_outcome_integrate.zip -d $cache_dir
!unzip -oq test7a_outcome_integrate.zip -d $cache_dir
!unzip -oq test7b_outcome_integrate.zip -d $cache_dir
!unzip -oq test7c_outcome_integrate.zip -d $cache_dir
!unzip -oq test7d_outcome_integrate.zip -d $cache_dir
!unzip -oq test7e_outcome_integrate.zip -d $cache_dir
!unzip -oq test7f_outcome_integrate.zip -d $cache_dir
!unzip -oq test7g_outcome_integrate.zip -d $cache_dir
!unzip -oq test7h_outcome_integrate.zip -d $cache_dir
!unzip -oq test7i_outcome_integrate.zip -d $cache_dir
!unzip -oq test7j_outcome_integrate.zip -d $cache_dir
!unzip -oq test7k_outcome_integrate.zip -d $cache_dir
!unzip -oq test7l_outcome_integrate.zip -d $cache_dir
!unzip -oq test7m_outcome_integrate.zip -d $cache_dir
!unzip -oq test7n_outcome_integrate.zip -d $cache_dir
!unzip -oq test7o_outcome_integrate.zip -d $cache_dir
!unzip -oq test7p_outcome_integrate.zip -d $cache_dir
!unzip -oq test7q_outcome_integrate.zip -d $cache_dir
!unzip -oq test7r_outcome_integrate.zip -d $cache_dir
!unzip -oq test7s_outcome_integrate.zip -d $cache_dir
!unzip -oq test7t_outcome_integrate.zip -d $cache_dir
!unzip -oq test8a_outcome_integrate.zip -d $cache_dir
!unzip -oq test8b_outcome_integrate.zip -d $cache_dir
!unzip -oq test8c_outcome_integrate.zip -d $cache_dir
!unzip -oq test8d_outcome_integrate.zip -d $cache_dir
!unzip -oq test8e_outcome_integrate.zip -d $cache_dir
!unzip -oq test8f_outcome_integrate.zip -d $cache_dir
!unzip -oq test8g_outcome_integrate.zip -d $cache_dir
!unzip -oq test8h_outcome_integrate.zip -d $cache_dir
!unzip -oq test2a_outcome_integrate.zip -d $cache_dir
!unzip -oq test2b_outcome_integrate.zip -d $cache_dir
!unzip -oq emma_11_30s_integrate_remedy.zip -d $cache_dir
!unzip -oq nb_lqx_29s_case2_integrate_remedy.zip -d $cache_dir
!unzip -oq nb_lqx_29s_integrate_remedy.zip -d $cache_dir
#all:59

In [ ]:
#test
import tensorflow as tf
import os
import re
from docx import Document

# Define paths for all cases
cases = [
    {"anchor": 'autodl-tmp/emma_11_30s_integrate_remedy', "positive":'autodl-tmp/nb_lqx_29s_case2_integrate2_remedy'},
    {"anchor": 'autodl-tmp/test2a_outcome_integrate', "positive": 'autodl-tmp/test2b_outcome_integrate'},
    
    {"anchor": 'autodl-tmp/test3d_outcome_integrate', "positive": 'autodl-tmp/test3e_outcome_integrate'},
    {"anchor": 'autodl-tmp/test3d_outcome_integrate', "positive": 'autodl-tmp/test3g_outcome_integrate'},
    {"anchor": 'autodl-tmp/test3d_outcome_integrate', "positive": 'autodl-tmp/test3h_outcome_integrate'},
    
    {"anchor": 'autodl-tmp/test4c_outcome_integrate', "positive": 'autodl-tmp/test4d_outcome_integrate'},    
    {"anchor": 'autodl-tmp/test4e_outcome_integrate', "positive": 'autodl-tmp/test4f_outcome_integrate'},
    
    {"anchor": 'autodl-tmp/test5a_outcome_integrate', "positive": 'autodl-tmp/test5b_outcome_integrate'},
    {"anchor": 'autodl-tmp/test5c_outcome_integrate', "positive": 'autodl-tmp/test5d_outcome_integrate'},
    {"anchor": 'autodl-tmp/test5g_outcome_integrate', "positive": 'autodl-tmp/test5h_outcome_integrate'},
    {"anchor": 'autodl-tmp/test5k_outcome_integrate', "positive": 'autodl-tmp/test5l_outcome_integrate'},
    
    {"anchor": 'autodl-tmp/test6a_outcome_integrate', "positive": 'autodl-tmp/test6c_outcome_integrate'},
    {"anchor": 'autodl-tmp/test6e_outcome_integrate', "positive": 'autodl-tmp/test6f_outcome_integrate'},
    {"anchor": 'autodl-tmp/test6g_outcome_integrate', "positive": 'autodl-tmp/test6h_outcome_integrate'},
    {"anchor": 'autodl-tmp/test6i_outcome_integrate', "positive": 'autodl-tmp/test6j_outcome_integrate'},
    {"anchor": 'autodl-tmp/test6k_outcome_integrate', "positive": 'autodl-tmp/test6l_outcome_integrate'},
    
    {"anchor": 'autodl-tmp/test7a_outcome_integrate', "positive": 'autodl-tmp/test7b_outcome_integrate'},
    {"anchor": 'autodl-tmp/test7c_outcome_integrate', "positive": 'autodl-tmp/test7d_outcome_integrate'},
    {"anchor": 'autodl-tmp/test7e_outcome_integrate', "positive": 'autodl-tmp/test7f_outcome_integrate'},
    {"anchor": 'autodl-tmp/test7g_outcome_integrate', "positive": 'autodl-tmp/test7h_outcome_integrate'},
    {"anchor": 'autodl-tmp/test7i_outcome_integrate', "positive": 'autodl-tmp/test7j_outcome_integrate'},
    {"anchor": 'autodl-tmp/test7k_outcome_integrate', "positive": 'autodl-tmp/test7l_outcome_integrate'},
    {"anchor": 'autodl-tmp/test7m_outcome_integrate', "positive": 'autodl-tmp/test7n_outcome_integrate'},
    {"anchor": 'autodl-tmp/test7o_outcome_integrate', "positive": 'autodl-tmp/test7p_outcome_integrate'},
    {"anchor": 'autodl-tmp/test7q_outcome_integrate', "positive": 'autodl-tmp/test7r_outcome_integrate'},
    {"anchor": 'autodl-tmp/test7s_outcome_integrate', "positive": 'autodl-tmp/test7t_outcome_integrate'},
    
    {"anchor": 'autodl-tmp/test8a_outcome_integrate', "positive": 'autodl-tmp/test8b_outcome_integrate'},
    {"anchor": 'autodl-tmp/test8c_outcome_integrate', "positive": 'autodl-tmp/test8d_outcome_integrate'},
    {"anchor": 'autodl-tmp/test8e_outcome_integrate', "positive": 'autodl-tmp/test8f_outcome_integrate'},
    {"anchor": 'autodl-tmp/test8g_outcome_integrate', "positive": 'autodl-tmp/test8h_outcome_integrate'}
    
    # Add more cases as needed
]


# Function to sort file names numerically
def numeric_sort(filenames):
    return sorted(filenames, key=lambda x: int(re.findall(r'\d+', x)[0]))



# Function to process a single case
def process_case(anchor_folder, positive_folder, case_name, window_size=5, step=1, threshold=1.8):
    # Load, filter, and sort image file names numerically
    anchor_images = numeric_sort([f for f in os.listdir(anchor_folder) if f.endswith(('.jpg', '.png'))])
    positive_images = numeric_sort([f for f in os.listdir(positive_folder) if f.endswith(('.jpg', '.png'))])

    # Convert to full paths
    anchor_images = [os.path.join(anchor_folder, f) for f in anchor_images]
    positive_images = [os.path.join(positive_folder, f) for f in positive_images]

    # Generate overlapping groups
    #anchor_groups = generate_overlapping_groups(anchor_images, window_size, step)
    #positive_groups = generate_overlapping_groups(positive_images, window_size, step)

    # Ensure the output directory exists
    output_dir = './outputnew_18k_epoch20_train1'
    os.makedirs(output_dir, exist_ok=True)

    # Create a new Document for the current case
    doc = Document()
    doc.add_heading(f'Testing Results: {case_name}', 0)

    start_index_p = 0
    start_index_a = 0
    # Start testing
    for j,positive_path in enumerate(positive_images):
        for i,anchor_path in enumerate(anchor_images):
            
            doc.add_heading(f'Testing Positive Image {j+1} with Anchor Image {i+1}', level=1)
            print (f'Testing Positive Image {j+1} with Anchor Image {i+1}')
           
            anchor_image = tf.image.decode_jpeg(tf.io.read_file(anchor_path), channels=3)
            anchor_image = tf.image.resize(anchor_image, [200, 200])
            anchor_image = resnet.preprocess_input(anchor_image)
            anchor_image = tf.expand_dims(anchor_image, axis=0)  # Add batch dimension

            positive_image = tf.image.decode_jpeg(tf.io.read_file(positive_path), channels=3)
            positive_image = tf.image.resize(positive_image, [200, 200])
            positive_image = resnet.preprocess_input(positive_image)
            positive_image = tf.expand_dims(positive_image, axis=0)
        
            # Generate embeddings
            embedding1 = embedding(resnet.preprocess_input(positive_image))
            embedding2 = embedding(resnet.preprocess_input(anchor_image))

            # Calculate Euclidean distance
            distance1 = tf.norm(embedding1 - embedding2, ord='euclidean').numpy()

                # Add to document
            doc.add_paragraph(f'Predicted Euclidean Distance: {distance1}')
            print(f'Predicted Euclidean Distance: {distance1}')
                
            # Save document for the current anchor-positive pair
            doc_name = os.path.join(output_dir, f'{case_name}_results.docx')
            doc.save(doc_name)
            #print(f"Results for {case_name} saved as {doc_name}")

# Iterate through all cases
for case in cases:
    case_name = f"{os.path.basename(case['anchor']).split('_')[0]}_vs_{os.path.basename(case['positive']).split('_')[0]}"
    print(f"Processing case: {case_name}")
    process_case(case["anchor"], case["positive"], case_name)